In [ ]:
# 00_process_data.ipynb

"""
This notebook processes raw Berlin transport data from Fahrplanbücher into structured formats.
"""

import sys
from pathlib import Path
import pandas as pd
import logging

# Add the src directory to the Python path
sys.path.append(str(Path('../src').resolve()))

# Import processing modules
from utils.data_loader import DataLoader, format_line_list
from processor import TransportDataProcessor
from db_station_matcher import Neo4jStationMatcher
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Berlin Transport Data Processing

This notebook performs the initial extraction and transformation of Berlin's historical public transportation data from raw sources. It represents the first step in our processing pipeline.

## Purpose

1. **Data Extraction**: Load and parse raw data from digitized Fahrplanbücher (timetables)
2. **Initial Structuring**: Convert raw data into structured tables with consistent formats
3. **Station Identification**: Establish unique identifiers for transportation stops
4. **Preliminary Geolocation**: Match stations to known geographic coordinates where possible

## Process Overview

The process follows these key steps:
1. Load raw data from CSV files containing transcribed Fahrplanbuch information
2. Process this data into standardized tables (lines and stops)
3. Match stations with existing station records to obtain geographic coordinates
4. Generate interim data files for subsequent processing stages

## Historical Context

The data represents Berlin's public transportation system during the Cold War era (1945-1989). During this period, Berlin was divided, with separate transportation authorities operating in East and West Berlin. This division is reflected in our data processing approach, where we handle each side separately for each year.

In [ ]:
# Configuration
YEAR = 1965
SIDE = "east"  # or "east"
DATA_DIR = Path('../data')

# Initialize loader
loader = DataLoader()

In [ ]:
# Load raw transcribed data
raw_data_path = DATA_DIR / 'raw' / f'{YEAR}_{SIDE}.csv'
raw_df = loader.load_raw_data(str(raw_data_path))
logger.info(f"Loaded raw data: {len(raw_df)} lines")

# Display sample of loaded data to verify
print("\nSample of loaded data:")
print(raw_df[['line_name', 'type', 'stops']].head())

## Existing Station Reference Data

To ensure consistency across years and facilitate geolocation, we maintain a reference dataset of known stations. This dataset:

1. Serves as a lookup table for station coordinates
2. Helps standardize station names across different time periods
3. Provides unique identifiers for stations that persist across snapshots
4. Records the lines that serve each station through time

As we process new data, this reference dataset will be expanded with newly identified stations.

In [ ]:
# Initialize Neo4j station matcher
neo4j_matcher = Neo4jStationMatcher(
    uri="bolt://localhost:7687",
    username="neo4j",
    password="BerlinTransport2024"
)

# Get existing stations from Neo4j
existing_stations_df = neo4j_matcher.get_all_stations()
logger.info(f"Loaded existing stations from Neo4j: {len(existing_stations_df)} stations")

## Initial Data Processing

The TransportDataProcessor class transforms our raw data into structured tables:

1. **Lines Table**: Contains information about each transportation line
   - Unique identifiers
   - Type (U-Bahn, S-Bahn, tram, bus)
   - Terminal stations
   - Service frequency
   - Journey time and distance

2. **Stops Table**: Contains information about each station
   - Unique identifiers
   - Station names
   - Transportation type
   - Placeholder for geographic coordinates

This structured format facilitates network analysis and visualization in later stages.

In [ ]:
# Process cleaned raw data
processor = TransportDataProcessor(YEAR, SIDE)

try:
    # Pass the DataFrame directly
    results = processor.process_raw_data(raw_df, existing_stations_df)
    logger.info("Initial processing complete")
    
    # Display processing results
    for name, df in results.items():
        print(f"\n{name} table shape: {df.shape}")
        print(f"Sample of {name}:")
        display(df.head(2))  # Using display for better notebook output
        
except Exception as e:
    logger.error(f"Error in initial processing: {e}")
    raise

## Station Matching

This step attempts to match stations in our current dataset with those in our reference database. This process:

1. Compares station names and types to find potential matches
2. Assigns geographic coordinates from matched stations
3. Identifies stations that require manual geolocation
4. Logs matching statistics for quality control

Stations that cannot be automatically matched will be processed manually using OpenRefine in a subsequent step.

In [ ]:
matcher = neo4j_matcher

# Process stops table with location matching
matched_stops = matcher.add_location_data(results['stops'])

# Don't forget to close the Neo4j connection when done
matcher.close()

In [ ]:
# Analysis of matching results
total_stops = len(matched_stops)
matched = matched_stops['location'].notna().sum()
unmatched = total_stops - matched

print("\nMatching Statistics:")
print(f"Total stations: {total_stops}")
print(f"Matched: {matched} ({matched/total_stops*100:.1f}%)")
print(f"Unmatched: {unmatched} ({unmatched/total_stops*100:.1f}%)")

# Display sample of matched stations
print("\nSample of matched stations:")
display(matched_stops[matched_stops['location'].notna()].head(3))

print("\nSample of unmatched stations:")
display(matched_stops[matched_stops['location'].isna()].head(3))

## Validation and Export

As a final step, we validate the matched stations and export the results:

1. The complete dataset is saved for the next processing stage
2. Unmatched stations are exported separately for manual geolocation
3. Matching statistics are logged for quality assurance

The manual geolocation process will be performed using OpenRefine, which provides tools for interactive data cleaning and enrichment.

In [ ]:
# Validate matches
from utils.geolocation import validate_matches
validate_matches(matched_stops)

# Save results
matched_dir = Path('../data/interim/stops_matched_initial')
matched_dir.mkdir(parents=True, exist_ok=True)

# Save all stops (both matched and unmatched)
matched_path = matched_dir / f'stops_{YEAR}_{SIDE}.csv'
matched_stops.to_csv(matched_path, index=False)

# Save unmatched stops separately for OpenRefine
unmatched_stops = matched_stops[matched_stops['location'].isna()]
openrefine_dir = Path('../data/interim/stops_for_openrefine')
openrefine_dir.mkdir(parents=True, exist_ok=True)
openrefine_path = openrefine_dir / f'unmatched_stops_{YEAR}_{SIDE}.csv'
unmatched_stops.to_csv(openrefine_path, index=False)

print(f"\nSaved {len(matched_stops)} total stops")
print(f"Exported {len(unmatched_stops)} unmatched stops for manual processing")

## Next Steps

After this initial processing, the workflow continues with:

1. **Manual Geolocation**: Using OpenRefine to add coordinates to unmatched stations
2. **Geolocation Verification**: Validating coordinates and splitting composite stations
3. **Data Enrichment**: Adding administrative and contextual information
4. **Network Construction**: Building a graph representation of the transportation system
5. **Analysis**: Investigating network properties and evolution over time

The next notebook in the sequence is `01_geolocation_verification_splitting.ipynb`.